In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("vine_filter").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [4]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ58W7SMO911M|          4|            2|          3|   N|                N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|                N|
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
| RATOTLA3OF70O|          5|            2|          2|   N|                N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|                N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|                N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|                N|
|R2XIM9LT335WHE|          4|            3|         10|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|     

In [26]:
# Establish percent helpful
helpful_df = vine_df.withColumn("percent_helpful", (vine_df['helpful_votes'] / vine_df['total_votes'])*100).filter("percent_helpful>50")
helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|  percent_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
| RQ58W7SMO911M|          4|            2|          3|   N|                N|66.66666666666666|
|  RF6IUKMGL8SF|          3|            5|          5|   N|                N|            100.0|
| RATOTLA3OF70O|          5|            2|          2|   N|                N|            100.0|
|R2F53LI9KK9MOY|          4|            2|          2|   N|                N|            100.0|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|                N|81.81818181818183|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|             80.0|
|R21SYDQ70ILUC0|          5|            1|          1|   N|                Y|            100.0|
| R7M06Z88PD7SX|          4|           1

In [27]:
# Filter total_votes column if >= 20
filtered_vine_df = helpful_df.filter("total_votes>=20")
filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|  percent_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|             80.0|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|67.74193548387096|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|96.42857142857143|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|            100.0|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|87.29641693811075|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|88.88888888888889|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|95.83333333333334|
|R2JSWNTAO83X3V|          5|          14

In [28]:
vine_program_df = filtered_vine_df.filter("vine == 'Y'")
vine_program_df.show()

+---------+-----------+-------------+-----------+----+-----------------+---------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percent_helpful|
+---------+-----------+-------------+-----------+----+-----------------+---------------+
+---------+-----------+-------------+-----------+----+-----------------+---------------+



In [29]:
non_vine_program_df = filtered_vine_df.filter("vine == 'N'")
non_vine_program_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|  percent_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|             80.0|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|67.74193548387096|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|96.42857142857143|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|            100.0|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|87.29641693811075|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|88.88888888888889|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|95.83333333333334|
|R2JSWNTAO83X3V|          5|          14

In [30]:
# Count of non-vine program review
non_vine_count = non_vine_program_df.count()
print(non_vine_count)

399745


In [31]:
# Number of non-vine program 5-star reviews
non_vine_5_count = non_vine_program_df.filter("star_rating=5").count()
print(non_vine_5_count)

241836


In [32]:
# Percentage of non-vine program 5-star reviews
percentage_non_vine_5_count = (non_vine_5_count / non_vine_count) * 100
print(percentage_non_vine_5_count)

60.49756719908942


In [33]:
# Count of vine program review
vine_count = vine_program_df.count()
print(vine_count)

0


In [34]:
# Number of vine program 5-star reviews
vine_5_count = vine_program_df.filter("star_rating=5").count()
print(vine_5_count)

0


In [35]:
# Percentage of vine program 5-star reviews